# Initialization #

In [2]:
from __future__ import division, print_function

import os
import bilby
import numpy as np

In [3]:
from Initial_data import InitialData
from Injection import InjectionSignal
from Subtraction import SubtractionSignal
from Projection import ProjectionSignal
from ORF_OF import detector_functions
from Cross_Correlation import CrossCorrelation
from plot_data import PSDWelch
from astro_stochastic_background import StochasticBackground
from CSD_Sum import SumCrossCorr

Injections directory already exist
Subtractions directory already exist
Projections directory already exist
ORF_OF directory already exist
Cross_Corr directory already exist
Plot_PSD directory already exist
Omega_gw directory already exist


__Specification of the output directory__

In [4]:
outdir = 'outdir'

bilby.utils.setup_logger(outdir=outdir, label='injected_signal')

if os.path.exists('./outdir'):
    print("outdir directory already exist")
else :
    print("outdir directory does not exist")
    try:
        os.mkdir(outdir)
    except OSError:
        print("Creation of the directory {} failed".format(outdir))
    else:
        print("Successfully created the directory {}".format(outdir))

if os.path.exists('./Sum'):
    print("Sum directory already exist")
else :
    print("Sum directory does not exist")
    try:
        os.mkdir('Sum')
    except OSError:
        print("Creation of the directory Sum failed")
    else:
        print("Successfully created the directory Sum")

outdir directory already exist
Sum directory already exist


__GW detector settings__

In [5]:
ifos = ['CE', 'ET_D_TR'] #, 'L1', 'H1', 'V1', 'K1']
sampling_frequency = 2048
start_time = 1198800017
end_time   = 1230336017
n_seg = 10000
Tfft = 8
# parameters to be included in the Fisher matrix (others were kept fixed at posterior sampling)
parameters = ['mass_1', 'mass_2', 'theta_jn', 'luminosity_distance']

__Initialization of data__

In [6]:
data = InitialData()
data_sets = data.initial_data(ifos, sampling_frequency, start_time, end_time, Tfft, n_seg)
sampling_frequency = data_sets[1]
start_time = data_sets[2]
end_time = data_sets[3]
duration_seg = data_sets[5]
N_samples = data_sets[7]
frequency = data_sets[8]
waveform_generator = data_sets[9]
IFOs = data_sets[10]

function =  detector_functions()
plots_PSD = PSDWelch()
cross_corr = CrossCorrelation()
Omega_gw = StochasticBackground()

function.initial(ifos,sampling_frequency, start_time, end_time, Tfft, n_seg)
Omega_gw.initial(ifos, sampling_frequency, start_time, end_time, Tfft, n_seg)

# Basic spectral analyses #

__ORF and optimal filter__

In [7]:
gamma = function.overlap_reduction_function()
optimal_filter = function.optimal_filter_JH(gamma)
function.plot_orf(gamma)
function.plot_optfilter(optimal_filter)

Antenna responses calculated.
Overlap-reduction function between CE and ET1 calculated.
Overlap-reduction function between CE and ET2 calculated.
Overlap-reduction function between CE and ET3 calculated.
Overlap-reduction function between ET1 and ET2 calculated.
Overlap-reduction function between ET1 and ET3 calculated.
Overlap-reduction function between ET2 and ET3 calculated.


/home/jan.harms/CBC_Projection/ORF_OF.py:265: RuntimeWarning: divide by zero encountered in true_divide
  Sb = np.dot((3. * self.H0**2) / (10 * np.pi**2), np.divide(self.omega_gw, self.frequency**3))
/home/jan.harms/CBC_Projection/ORF_OF.py:274: RuntimeWarning: divide by zero encountered in true_divide
  optimal_filter_JH[d1, d2, :] = ((gamma[d1, d2, :] * Sb) / (psd[d1, :] * psd[d2, :]))


__PSD of GW detectors__

In [8]:
psd_detector = function.psd()

__Stochastic backgrounds and plot__

In [9]:
# ## Astro and Cosmological Backgrounds
# astro_omega_gw = Omega_gw.astro_Omega_gw()
# cosmo_omega_gw = Omega_gw.cosmo_Omega_gw()

## Cosmological Spectrum and COBE Spectrum
cosmo_spectrum = Omega_gw.cosmo_spectrum()
cobe_spectrum = Omega_gw.cobe_spectrum()
PGWB = Omega_gw.PGWB()
plots_PSD.plot_comso_omega_gw(frequency, comso_omega_gw=cosmo_spectrum, cobe_spectrum=cobe_spectrum, PGWB=PGWB)
plot_all_gw = Omega_gw.plot_all_backgrounds()

__CSD from Omega_gw__

In [10]:
cross_corr.initial(ifos, sampling_frequency, start_time, end_time, n_seg, gamma=gamma, optimal_filter=optimal_filter, Tfft=Tfft)
## CSD from Omega_GW of Astrophysical Origin
CSD_from_Omega_astro = cross_corr.CSD_from_Omega_astro()
# print('CSD_from_Omega_astro',CSD_from_Omega_astro)
plot_CSD_from_omega = plots_PSD.plot_CSD_from_Omega(IFOs, frequency, CSD_from_Omega=CSD_from_Omega_astro, background='Astro')
## CSD from Omega_GW of Cosmological Origin
CSD_from_Omega_cosmo = cross_corr.CSD_from_Omega_cosmo()
# print('CSD_from_Omega_cosmo',CSD_from_Omega_cosmo)
plot_CSD_from_omega = plots_PSD.plot_CSD_from_Omega(IFOs, frequency, CSD_from_Omega=CSD_from_Omega_cosmo, background='Cosmo')

/home/jan.harms/CBC_Projection/Cross_Correlation.py:143: RuntimeWarning: invalid value encountered in true_divide
  Sb = np.dot((3. * self.H0**2) / (10 * np.pi**2), np.divide(omega_gw, self.frequency**3))
/home/jan.harms/CBC_Projection/Cross_Correlation.py:165: RuntimeWarning: divide by zero encountered in true_divide
  Sb = np.dot((3. * self.H0**2) / (10 * np.pi**2), np.divide(self.omega_gw, self.frequency**3))


# Simulation of CBC foreground and CSDs of original h(t), subtraction residuals, and projected data #

In [11]:
## Number of Binary Signals
#n_inj = 100
n_inj = 10
## Number of Time segments
#n_seg = 100
n_seg = 1

  
injection = InjectionSignal()
injection_params = injection.injection_parameters(filename='./Signals/injections_10e6.hdf5')
subtraction = SubtractionSignal()
bestfit_params = subtraction.subtraction_parameters(filename='./Signals/best_fit_params.pkl')

## index of signals with bad PE
nw = [8,85,86,87,94,102,103]

injection_params = injection_params.drop(nw)
bestfit_params = bestfit_params.drop(nw)
injection_params = injection_params.loc[0:n_inj-1]
bestfit_params = bestfit_params.loc[0:n_inj-1]

projection = ProjectionSignal()

#print(injection_params[53:55])
#print(bestfit_params[53:55])
# print('injection_param_redshift',injection_param['redshift'])
# print('bestfit_params',bestfit_params[100:101])

## Defining sum arrays for CSD to take sum over all time segment
n_det = len(IFOs)
sum_noise = np.zeros((n_det, n_det, len(frequency)), dtype = np.complex)
sum_inj = np.zeros((n_det, n_det, len(frequency)), dtype = np.complex)
sum_sub = np.zeros((n_det, n_det, len(frequency)), dtype = np.complex)
sum_proj = np.zeros((n_det, n_det, len(frequency)), dtype = np.complex)
variance = np.zeros((n_det, n_det, len(frequency)))

__Variance in PSD__

In [12]:
"""Variance defined over the Gaussian power spectral densities for each detector pair and normalized over
the number of fft samples for each time segment and total number of time segments."""
cidx = 0
for d1 in np.arange(n_det):
    detector_1 = psd_detector[d1, :]
    for d2 in range(d1+1, n_det):
        detector_2 = psd_detector[d2, :]
        variance[d1,d2, :] = np.sqrt(detector_1 * detector_2) / np.sqrt(2 * duration_seg / Tfft * n_seg)

# plot_variance = plots_PSD.plot_variance(IFOs, frequency, variance, n_seg=None, method='noise_'+str(n_seg))

__Injection-Subtraction-Projection__

In [ ]:
for k in np.arange(n_seg):

    print('Data segment: ',100*(k+1)/n_seg,'%')

    ## Setting the seg_start_time and seg_end_time
    seg_start_time = start_time + k * duration_seg
    seg_end_time = seg_start_time + duration_seg

    ## Initializing the Interferometer
    IFOs = bilby.gw.detector.networks.InterferometerList(ifos)

    ## One can choose where you want to Inject and Subtract your Binary Signals.
    ## Choice:1 Setting strain data from zero noise
    IFOs.set_strain_data_from_zero_noise(sampling_frequency,duration=duration_seg,start_time=start_time)
    ## Choice:2 Setting strain data from power spectral density of the detector.
    # IFOs.set_strain_data_from_power_spectral_densities(sampling_frequency=sampling_frequency,duration=duration_seg,
    #                                                    start_time=seg_start_time)

    ## Time series for each detector
    noise_time_series = np.zeros((n_det, N_samples))
    ci = 0
    for ifo in IFOs:
        noise_time_series[ci, :] = ifo.strain_data.time_domain_strain
        ci += 1

    ##################################
    # injection Parameter and Signal #
    ##################################

    t_coalescence, inj_time_series, IFOs = injection.injection_signal(IFOs, sampling_frequency,
                            seg_start_time, seg_end_time, k, N_samples, injection_params, waveform_generator)

    print('injected time series is',inj_time_series)


    ####################################
    # Subtraction Parameter and Signal #
    ####################################

    sub_time_series, IFOs = subtraction.subtraction_signal(IFOs, sampling_frequency,
                          seg_start_time, seg_end_time, t_coalescence, k, N_samples, bestfit_params, waveform_generator)

    print('subtract time series is', sub_time_series)


    ##############################################
    # Projection Parameter Derivative and Signal #
    ##############################################

    proj_time_series = projection.projection_signal(IFOs, sampling_frequency, 
                        seg_start_time, seg_end_time, t_coalescence, k, N_samples, bestfit_params, waveform_generator, sub_time_series, parameters=parameters)

    print('projected time series is', proj_time_series)

    ###################
    # PSD_Welch Plots #
    ###################

    # ## For one case
    # plot_data = plots_PSD.plot_one_psd(IFOs, sampling_frequency, k, inj_time_series, Tfft)

    ## For injection,subtraction and projection together
    plot_data = plots_PSD.plot_data(IFOs, sampling_frequency, k, inj_time_series, sub_time_series, proj_time_series, Tfft)

    ###################################################################
    # Cross-Correlation between the time series for different signals #
    ###################################################################

    """"### Cross-Correlation for Noise ###"""
    cc_noise, cc_noise_filt = cross_corr.cross_correlation(noise_time_series)
    cross_corr.plot_cc(cc_noise, k, 'noise')

    """Taking sum over all measurement for noise cross-correlation i.e. cc_noise to calculate the mean deviation."""
    for d1 in np.arange(n_det):
        for d2 in np.arange(d1+1,n_det):
            sum_noise[d1,d2, :] += cc_noise[d1,d2, :]/n_seg

    """"### Cross-Correlation for Injections ###"""
    cc_inj, cc_inj_filt = cross_corr.cross_correlation(inj_time_series)
    cross_corr.plot_cc(cc_inj, k, 'injected_signal')

    """Taking sum over all measurement for injection cross-correlation i.e. cc_noise to calculate the mean deviation."""
    for d1 in np.arange(n_det):
        for d2 in np.arange(d1+1,n_det):
            sum_inj[d1,d2, :] += cc_inj[d1,d2, :]/n_seg

    """ ### Cross-Correlation for Subtractions ### """
    cc_sub, cc_sub_filt = cross_corr.cross_correlation(sub_time_series)
    cross_corr.plot_cc(cc_sub, k, 'subtracted_signal')

    """Taking sum over all measurement for subtraction cross-correlation i.e. cc_noise to calculate the mean deviation."""
    for d1 in np.arange(n_det):
        for d2 in np.arange(d1+1,n_det):
            sum_sub[d1,d2, :] += cc_sub[d1,d2, :]/n_seg

    """Cross-Correlation for Projections"""
    cc_proj, cc_proj_filt = cross_corr.cross_correlation(proj_time_series)
    cross_corr.plot_cc(cc_proj, k, 'projected_signal')

    """Taking sum over all measurement for projection cross-correlation i.e. cc_noise to calculate the mean deviation."""
    for d1 in np.arange(n_det):
        for d2 in np.arange(d1+1,n_det):
            sum_proj[d1,d2, :] += cc_proj[d1,d2, :]/n_seg

    ############
    # Plot CSD #
    ############

    plot_CSD = PSDWelch()
    plot_csd = plot_CSD.plots_csd(IFOs, frequency, psd_series=psd_detector, cc_inj=cc_inj, cc_sub=cc_sub, cc_proj=cc_proj, n_seg=k)

## Plot sum of Projection time series with Cross-Correlation, Variance, CSD from astro and cosmo backgrounds
plot_csd_var = plots_PSD.plot_csd_var(IFOs, frequency, sum_series=sum_noise, variance=variance, CSD_from_Omega_astro=CSD_from_Omega_astro,
                                    CSD_from_Omega_cosmo=CSD_from_Omega_cosmo, n_seg=k, method='Noise')

## Plot sum of Projection time series with Cross-Correlation, Variance, CSD from astro and cosmo backgrounds
plot_csd_var = plots_PSD.plot_csd_var(IFOs, frequency, sum_series=sum_inj, variance=variance, CSD_from_Omega_astro=CSD_from_Omega_astro,
                                    CSD_from_Omega_cosmo=CSD_from_Omega_cosmo, n_seg=k, method='Injections')
        
## Plot sum of Projection time series with Cross-Correlation, Variance, CSD from astro and cosmo backgrounds
plot_csd_var = plots_PSD.plot_csd_var(IFOs, frequency, sum_series=sum_sub, variance=variance,
                                    CSD_from_Omega_astro=CSD_from_Omega_astro,CSD_from_Omega_cosmo=CSD_from_Omega_cosmo, n_seg=k, method='Subtraction')

## Plot sum of Projection time series with Cross-Correlation, Variance, CSD from astro and cosmo backgrounds
plot_csd_var = plots_PSD.plot_csd_var(IFOs, frequency, sum_series=sum_proj, variance=variance,
                                    CSD_from_Omega_astro=CSD_from_Omega_astro,CSD_from_Omega_cosmo=CSD_from_Omega_cosmo, n_seg=k, method='Projection')

Data segment:  100.0 %
Injection script starting ( 147.895833408 )
Number of Injection signal is : 0
seg_start_time 1198800017
geocent_time 1198801970.420822
seg_end_time 1198804113


02:24 bilby INFO    : Injected signal in CE:
02:24 bilby INFO    :   optimal SNR = 4649.46
02:24 bilby INFO    :   matched filter SNR = 4649.46+0.00j
02:24 bilby INFO    :   a_1 = 0.7263597956111914
02:24 bilby INFO    :   a_2 = 0.40047642065704436
02:24 bilby INFO    :   tilt_1 = 1.6896867333637358
02:24 bilby INFO    :   tilt_2 = 1.2474190774518785
02:24 bilby INFO    :   phi_12 = 0.3743726669615049
02:24 bilby INFO    :   phi_jl = 6.06087770142716
02:24 bilby INFO    :   dec = -0.09344687603372186
02:24 bilby INFO    :   ra = 5.6418648232743775
02:24 bilby INFO    :   iota = 0.6816452193107659
02:24 bilby INFO    :   psi = 2.8696119231878674
02:24 bilby INFO    :   phase = 4.545929587458194
02:24 bilby INFO    :   redshift = 0.015213003592662058
02:24 bilby INFO    :   mass_1 = 11.25047842929217
02:24 bilby INFO    :   mass_2 = 7.295521493144897
02:24 bilby INFO    :   geocent_time = 1198801970.420822
02:24 bilby INFO    :   theta_jn = 0.6816452193107659
02:24 bilby INFO    :   lumi

Number of Injection signal is : 1
seg_start_time 1198800017
geocent_time 1198800251.7243042
seg_end_time 1198804113


02:25 bilby INFO    : Injected signal in CE:
02:25 bilby INFO    :   optimal SNR = 5789.15
02:25 bilby INFO    :   matched filter SNR = 5789.14-0.01j
02:25 bilby INFO    :   a_1 = 0.42024866697924956
02:25 bilby INFO    :   a_2 = 0.6136935257177568
02:25 bilby INFO    :   tilt_1 = 1.9050456090985612
02:25 bilby INFO    :   tilt_2 = 1.7151491274304782
02:25 bilby INFO    :   phi_12 = 4.490192739379045
02:25 bilby INFO    :   phi_jl = 2.9224182602573516
02:25 bilby INFO    :   dec = 0.02461953914939593
02:25 bilby INFO    :   ra = 5.375579225388474
02:25 bilby INFO    :   iota = 2.243572534118937
02:25 bilby INFO    :   psi = 2.367688412072135
02:25 bilby INFO    :   phase = 2.665091095387514
02:25 bilby INFO    :   redshift = 0.018893509639863672
02:25 bilby INFO    :   mass_1 = 56.19719756245719
02:25 bilby INFO    :   mass_2 = 9.219779915912051
02:25 bilby INFO    :   geocent_time = 1198800251.7243042
02:25 bilby INFO    :   theta_jn = 2.243572534118937
02:25 bilby INFO    :   luminos

Number of Injection signal is : 2
seg_start_time 1198800017
geocent_time 1198803669.2636516
seg_end_time 1198804113


...

Number of Subtraction signal is : 7
seg_start_time 1198800017
geocent_time 1198801312.545888
seg_end_time 1198804113


02:51 bilby INFO    : Subtracted signal in CE:
02:51 bilby INFO    :   optimal SNR = 3304.87
02:51 bilby INFO    :   matched filter SNR = -1.55+1.38j
02:51 bilby INFO    :   mass_1 = 43.28247568467698
02:51 bilby INFO    :   mass_2 = 7.766687528408846
02:51 bilby INFO    :   luminosity_distance = 113.06786477141874
02:51 bilby INFO    :   theta_jn = 2.4535296958665014
02:51 bilby INFO    :   a_1 = 0.37319937102168316
02:51 bilby INFO    :   a_2 = 0.554344856290986
02:51 bilby INFO    :   tilt_1 = 0.8588475520298485
02:51 bilby INFO    :   tilt_2 = 2.2529546374535223
02:51 bilby INFO    :   phi_12 = 0.7969201790720661
02:51 bilby INFO    :   phi_jl = 1.285488395956359
02:51 bilby INFO    :   dec = -0.4060910199791631
02:51 bilby INFO    :   ra = 0.6990461846470951
02:51 bilby INFO    :   psi = 1.3485998367624785
02:51 bilby INFO    :   phase = 4.477591233340565
02:51 bilby INFO    :   geocent_time = 1198801312.545888
02:51 bilby INFO    : Subtracted signal in ET1:
02:51 bilby INFO    : 

Number of Subtraction signal is : 9
seg_start_time 1198800017
geocent_time 1198802489.578122
seg_end_time 1198804113


02:52 bilby INFO    : Subtracted signal in CE:
02:52 bilby INFO    :   optimal SNR = 2085.32
02:52 bilby INFO    :   matched filter SNR = -0.57-1.48j
02:52 bilby INFO    :   mass_1 = 16.642388818188884
02:52 bilby INFO    :   mass_2 = 11.245045953819517
02:52 bilby INFO    :   luminosity_distance = 125.02692046253277
02:52 bilby INFO    :   theta_jn = 1.4483113704521564
02:52 bilby INFO    :   a_1 = 0.7762282886175819
02:52 bilby INFO    :   a_2 = 0.4177322113620727
02:52 bilby INFO    :   tilt_1 = 1.0389558136775086
02:52 bilby INFO    :   tilt_2 = 2.078080250434275
02:52 bilby INFO    :   phi_12 = 0.39314226317899353
02:52 bilby INFO    :   phi_jl = 5.7298186520247905
02:52 bilby INFO    :   dec = 0.5848972315338888
02:52 bilby INFO    :   ra = 1.0918165478609152
02:52 bilby INFO    :   psi = 0.221839870192886
02:52 bilby INFO    :   phase = 5.451972076089301
02:52 bilby INFO    :   geocent_time = 1198802489.578122
02:52 bilby INFO    : Subtracted signal in ET1:
02:52 bilby INFO    :

Signals subtracted ( 562.791261873 )
Subtraction plots saved ( 787.967251375 )
Residual time series calculated ( 790.184516603 )
subtract time series is [[ 1.51466795e-28  1.58619716e-28  1.65515279e-28 ...
   1.29536282e-28  1.36873877e-28  1.44345219e-28]
 [-3.78718520e-25 -3.74045878e-25 -3.69343860e-25 ...
  -3.92553166e-25 -3.87972579e-25 -3.83360733e-25]
 [ 7.01839596e-25  7.00812085e-25  6.99725555e-25 ...
   7.04566067e-25  7.03716502e-25  7.02807621e-25]
 [-3.23118835e-25 -3.26764148e-25 -3.30379817e-25 ...
  -3.12010120e-25 -3.15741322e-25 -3.19444466e-25]]
Projection script starting ( 790.186144221 )

Number of Projection signal is : 0
Signal derivatives calculated ( 840.889650911 )
Number of Projection signal is : 1
Signal derivatives calculated ( 880.610892279 )
Number of Projection signal is : 2
Signal derivatives calculated ( 936.40168277 )
Number of Projection signal is : 3
Signal derivatives calculated ( 991.450680348 )
Number of Projection signal is : 4
Signal derivat